# Pokemon Showdown Gym - Google Colab Setup

This notebook sets up the Pokemon Showdown Gym environment for reinforcement learning in Google Colab.

## Features
- Automatic installation of all dependencies
- Pokemon Showdown server setup
- Environment testing and validation
- Ready-to-use training examples


## 1. Install Dependencies

First, let's install all the required packages:


In [ ]:
# Install system dependencies
!apt-get update
!apt-get install -y nodejs npm

# Install Python packages (following README.md requirements)
!pip install -q poke-env==0.10.0
!pip install -q numpy==2.2.5
!pip install -q gymnasium
!pip install -q stable-baselines3
!pip install -q tensorboard
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q pandas
!pip install -q requests  # For server health checks

# Install additional packages mentioned in README.md
!pip install -q pipreqs  # For requirements.txt generation

print("✅ All packages installed successfully!")
print("📝 Following README.md installation requirements")


## 2. Clone and Setup Repository

Clone the showdown_gym repository and install it:


In [ ]:
# Clone the repository
!git clone https://github.com/UoA-CARES/showdown_gym.git

# Install the package in development mode
!cd showdown_gym && pip install -e .

print("✅ Repository cloned and installed!")


## 2.1. Install Additional Dependencies (README.md Requirements)

Install the additional packages mentioned in README.md:


In [ ]:
# Install cares_reinforcement_learning (as per README.md)
!git clone https://github.com/UoA-CARES/cares_reinforcement_learning.git
!cd cares_reinforcement_learning && pip install -r requirements.txt
!cd cares_reinforcement_learning && pip install -e .

# Install gymnasium_environments (as per README.md)
!git clone https://github.com/UoA-CARES/gymnasium_envrionments.git
!cd gymnasium_envrionments && pip install -r requirements.txt

print("✅ Additional dependencies installed!")
print("📝 Following README.md package installation order")


## 3. Setup Pokemon Showdown Server

Install and configure the Pokemon Showdown server (following README.md instructions):


In [ ]:
# Clone Pokemon Showdown server (following README.md instructions)
!git clone https://github.com/smogon/pokemon-showdown.git

# Install dependencies
!cd pokemon-showdown && npm install

# Copy configuration (as specified in README.md)
!cd pokemon-showdown && cp config/config-example.js config/config.js

print("✅ Pokemon Showdown server setup complete!")
print("📝 Note: Server must be running for environment to work (as per README.md)")


## 4. Start Pokemon Showdown Server

Start the server in the background:


In [ ]:
import subprocess
import time
import os
import threading
import requests

# Start the server in the background (following README.md instructions)
def start_showdown_server():
    os.chdir('/content/pokemon-showdown')
    process = subprocess.Popen(
        ['node', 'pokemon-showdown', 'start', '--no-security'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        preexec_fn=os.setsid
    )
    return process

# Function to check if server is running
def check_server_running():
    try:
        response = requests.get('http://localhost:8000', timeout=5)
        return response.status_code == 200
    except:
        return False

# Start server
print("🚀 Starting Pokemon Showdown server...")
server_process = start_showdown_server()

# Wait for server to start and verify it's running
print("⏳ Waiting for server to start...")
for i in range(30):  # Wait up to 30 seconds
    time.sleep(1)
    if check_server_running():
        print("✅ Pokemon Showdown server started successfully!")
        print("🌐 Server is running on http://localhost:8000")
        break
    print(f"⏳ Still waiting... ({i+1}/30)")
else:
    print("❌ Server failed to start within 30 seconds")
    print("💡 Try restarting the runtime and running this cell again")


## 5. Test Environment

Test that the environment is working correctly:


## 6.1. Test with README.md Training Command

Test the environment using the exact command from README.md:


In [ ]:
# Test with the exact command from README.md
# This follows the README.md instructions for testing the environment

import os
import sys

# Add the gymnasium_environments to path
sys.path.append('/content/gymnasium_envrionments')

# Test the environment using README.md command
print("🧪 Testing environment with README.md command...")
print("Command: python run.py train cli --gym showdown --domain random --task max DQN")

# Change to the gymnasium_environments directory
os.chdir('/content/gymnasium_envrionments/scripts')

# Run the test command from README.md
!python run.py train cli --gym showdown --domain random --task max DQN --display 0

print("✅ README.md training command test completed!")


In [ ]:
import sys
sys.path.append('/content/showdown_gym')

from showdown_gym.showdown_environment import SingleShowdownWrapper
import numpy as np

# Test environment creation
try:
    env = SingleShowdownWrapper(team_type="random", opponent_type="random")
    print("✅ Environment created successfully!")
    
    # Test environment reset
    obs = env.reset()
    print(f"✅ Environment reset successful!")
    print(f"Observation shape: {obs.shape}")
    print(f"Action space size: {env.action_space.n}")
    
    # Test a random action
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    print(f"✅ Step successful! Reward: {reward}")
    
except Exception as e:
    print(f"❌ Error testing environment: {e}")
    print("Make sure the Pokemon Showdown server is running!")


## 6. Training Example

Here's a simple example of training an agent:


In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
import matplotlib.pyplot as plt

# Create vectorized environment
def make_env():
    return SingleShowdownWrapper(team_type="random", opponent_type="random")

env = make_vec_env(make_env, n_envs=1)

# Create DQN agent
model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.001,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=32,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    target_update_interval=1000
)

print("✅ DQN model created successfully!")
print("You can now train the model with: model.learn(total_timesteps=10000)")
